# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions;
                      functionals=[:lda_x, :lda_c_vwn],
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770854281101                   -0.52    9.0    181ms
  2   -2.772145388521       -2.89       -1.32    1.0    128ms
  3   -2.772170300462       -4.60       -2.50    1.0    109ms
  4   -2.772170648148       -6.46       -3.25    1.0    112ms
  5   -2.772170721171       -7.14       -3.87    2.0    525ms
  6   -2.772170722844       -8.78       -4.52    1.0    111ms
  7   -2.772170723010       -9.78       -5.14    2.0    135ms
  8   -2.772170723015      -11.33       -6.29    1.0    149ms
  9   -2.772170723015      -12.98       -6.28    2.0    159ms
 10   -2.772170723015      -13.50       -7.03    1.0    119ms
 11   -2.772170723015      -14.75       -7.48    1.0    119ms
 12   -2.772170723015   +  -14.12       -8.34    1.0    128ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.7

1.773558033845541

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem. This leads to a density-functional perturbation
theory problem, which is automatically set up and solved in the background.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770760377912                   -0.52    9.0    187ms
  2   -2.772058989104       -2.89       -1.32    1.0    118ms
  3   -2.772082970364       -4.62       -2.44    1.0    116ms
  4   -2.772083329359       -6.44       -3.11    1.0    114ms
  5   -2.772083416831       -7.06       -3.98    2.0    132ms
  6   -2.772083417604       -9.11       -4.38    1.0    527ms
  7   -2.772083417807       -9.69       -5.60    1.0    113ms
  8   -2.772083417810      -11.44       -5.81    2.0    146ms
  9   -2.772083417811      -12.67       -6.26    1.0    134ms
 10   -2.772083417811      -13.39       -7.05    1.0    118ms
 11   -2.772083417811   +  -14.65       -7.59    1.0    119ms
 12   -2.772083417811   +  -13.99       -8.54    2.0    132ms

Polarizability via ForwardDiff:       1.7725349795236738
Polarizability via finite difference: 1.773558033845541
